In [165]:
import numpy as np
import random as rand
import pandas as pd
import math
import xlsxwriter as xw
from typing import Callable

In [166]:
def func_objetivo(x):
    return (2*x[0] - 2*x[1] + 6)*(2*x[0] - 2*x[1] + 6) + (-2*x[0] + 2*x[1] - 6)*(-2*x[0] + 2*x[1] - 6)

In [167]:
class Particle:
    def __init__(self, x0, num_dimensions):
        self.position_i = [x for x in x0]
        self.velocity_i = [rand.uniform(-1, 1) for _ in range(num_dimensions)]
        self.pos_best_i = [x for x in x0]
        self.err_best_i = -1
        self.err_i      = -1

    def evaluate_particle(self, cost_func):
        self.err_i = cost_func(self.position_i)
        if self.err_i < self.err_best_i or self.err_best_i == -1:
            self.pos_best_i = [x for x in self.position_i]
            self.err_best_i = self.err_i

    def attVel(self, pos_best_g, w, c1, c2):

        for i in range(len(self.velocity_i)):
            r1 = rand.random()
            r2 = rand.random()

            vel_cognitive = c1 * r1 * (self.pos_best_i[i] - self.position_i[i])
            vel_social = c2 * r2 * (pos_best_g[i] - self.position_i[i])
            self.velocity_i[i] = w * self.velocity_i[i] + vel_cognitive + vel_social

    def attPos(self, bounds):
        for i in range(len(self.position_i)):
            self.position_i[i] = self.position_i[i] + self.velocity_i[i]

            if self.position_i[i] > bounds[i][1]:
                self.position_i[i] = bounds[i][1]

            if self.position_i[i] < bounds[i][0]:
                self.position_i[i] = bounds[i][0]

In [168]:
def alg_pso(cost_func, x0, bounds, num_particles, max_iter, inercia, social, cognitivo, tol):
    num_dimensions = len(x0)
    err_best_g = math.inf
    pos_best_g = []
    swarm = [Particle(x0, num_dimensions) for _ in range(num_particles)]
    results = []

    for iteration in range(max_iter):
        for particle in swarm:
            particle.evaluate_particle(cost_func)
            if particle.err_i < err_best_g or math.isinf(err_best_g):
                pos_best_g = list(particle.pos_best_i)
                err_best_g = particle.err_i

        if err_best_g <= tol:
            results.append([pos_best_g, err_best_g])
            break

        for particle in swarm:
            particle.attVel(pos_best_g, inercia, social, cognitivo)
            particle.attPos(bounds)

        results.append([pos_best_g, err_best_g])

    return results

In [169]:
def test_function(num_tests:int, func:Callable, params:list, filename:str):
    wb = xw.Workbook(filename)
    ws = wb.add_worksheet("1")

    chart = wb.add_chart({'type':'scatter','subtype': 'straight_with_markers'})

    n_iters = []
    
    for test_number in range(num_tests):
        initial = [rand.uniform(-10, 10), rand.uniform(-10, 10)]
        results = func(params[0],
                       initial,
                       params[2],
                       params[3],
                       params[4],
                       params[5],
                       params[6],
                       params[7],
                       params[8])
        n_iters.append(len(results))
        ws.merge_range(0, 3*test_number, 0, 3*test_number + 2, test_number)
        ws.merge_range(1, 3*test_number, 1, 3*test_number + 2, len(results))
        ws.write_column(2, 3*test_number, [elem[0][0] for elem in results])
        ws.write_column(2, 3*test_number + 1, [elem[0][1] for elem in results])
        ws.write_column(2, 3*test_number + 2, [elem[1] for elem in results])
        if test_number < 50:
            chart.add_series({
                'categories': ['1', 2, 3*test_number, 2 + len(results), 3*test_number],
                'values':     ['1', 2, 3*test_number + 1, 2 + len(results), 3*test_number + 1],
                'marker': {'type': 'circle'}
            })
    avg = 0
    for iter in n_iters:
        avg += iter
    avg /= len(n_iters)
    print("qtd média de iterações:",avg)
    chart.set_size({'x_scale': 2, 'y_scale': 2})
    chart.set_legend({'none': True})
    chart.set_chartarea({'border': {'none': True}})
    ws.insert_chart(3+len(results), 0, chart)
    chart.set_x_axis({'major_gridlines': {'visible': True}})
    chart.set_y_axis({'major_gridlines': {'visible': True}})
    wb.close()

In [170]:
# Parâmetros do PSO
inercias = [0.4, 0.5, 0.6]
sociais = [1, 2]
cognitivos = [2, 3]
tol = 0.00001

# Condições iniciais
initial = [5, 5]
bounds = [(-100, 100), (-100, 100)]
num_particles = 15
max_iterations = 9999

# Execução do PSO
for inercia in inercias:
    for social in sociais:
        for cognitivo in cognitivos:
            print(inercia, social, cognitivo)
            test_function(
                1000,
                alg_pso,
                [func_objetivo,
                initial,
                bounds,
                num_particles,
                max_iterations,
                inercia,
                social,
                cognitivo,
                tol],
                f"teste_pso_i{inercia}_s{social}_c{cognitivo}.xlsx"
            )

0.5 1 2
qtd média de iterações: 18.473
0.5 1 3
qtd média de iterações: 28.567
0.5 2 2
qtd média de iterações: 22.108
0.5 2 3
qtd média de iterações: 121.077
0.65 1 2
qtd média de iterações: 24.722
0.65 1 3
qtd média de iterações: 51.505
0.65 2 2
qtd média de iterações: 36.328
0.65 2 3
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\lucas\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lucas\AppData\Local\Temp\ipykernel_28996\2100578376.py", line 18, in <module>
    test_function(
  File "C:\Users\lucas\AppData\Local\Temp\ipykernel_28996\2755510168.py", line 11, in test_function
    results = func(params[0],
              ^^^^^^^^^^^^^^^
  File "C:\Users\lucas\AppData\Local\Temp\ipykernel_28996\3412913390.py", line 21, in alg_pso
    particle.attPos(bounds)
  File "C:\Users\lucas\AppData\Local\Temp\ipykernel_28996\1496583199.py", line -1, in attPos
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\lucas\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structure